----TODO-----
Actualizar feronomonas, sobre todo fitness por pieza
Pruebas Fitness

In [1]:
import copy
#import numpy as np
from typing import List, Dict

In [2]:
class PiezaClase:
    def __init__(self, pieza, x_pos, y_pos):
        self.pieza = pieza
        self.x_pos = x_pos
        self.y_pos = y_pos
    
"""     def _initialize(pieza, x_pos, y_pos):
        self.pieza = pieza
        self.x_pos = x_pos
        self.y_pos = y_pos """

'     def _initialize(pieza, x_pos, y_pos):\n        self.pieza = pieza\n        self.x_pos = x_pos\n        self.y_pos = y_pos '

In [3]:
def explore_adyacent(temp, x, y): #hay que marcar la celda inicial antes de llamarla
    area = 0
    # Verificar si la celda adyacente está dentro de los límites del espacio
    if x + 1 < len(temp) and temp[x+1][y] == 0:
        temp[x+1][y] = 1
        area += explore_adyacent(temp, x+1, y)
    if x - 1 >= 0 and temp[x-1][y] == 0:
        temp[x-1][y] = 1
        area += explore_adyacent(temp, x-1, y)
    if y + 1 < len(temp[0]) and temp[x][y+1] == 0:
        temp[x][y+1] = 1
        area += explore_adyacent(temp, x, y+1)
    if y - 1 >= 0 and temp[x][y-1] == 0:
        temp[x][y-1] = 1
        area += explore_adyacent(temp, x, y-1)
    if temp[x][y]:
        area += 1
    return area


def huecos (mat):
  matrix = copy.deepcopy(mat)
  array = []
  for i in range(len(mat)):
    for j in range(len(mat[0])):
      if matrix[i][j] == 0:
        matrix[i][j] = 1
        array.append(explore_adyacent(matrix, i, j))
  return array

space =  [[0, 1, 0, 1],
          [0, 1, 0, 1],
          [0, 0, 1, 0],
          [0, 1, 1, 1]]
temp_area = copy.deepcopy(space)
array = huecos(temp_area)
print(space)
print (temp_area)
print (array)

[[0, 1, 0, 1], [0, 1, 0, 1], [0, 0, 1, 0], [0, 1, 1, 1]]
[[0, 1, 0, 1], [0, 1, 0, 1], [0, 0, 1, 0], [0, 1, 1, 1]]
[5, 2, 1]


In [4]:
def clamp_to_zero(number):
    if number < 0:
        return 0
    else:
        return number

FITNESS DEFINITION

In [5]:
def fitness(space, pieza, x, y):
  valid = colocar_pieza(space, pieza, x, y)
  espacios = huecos(space)
  if not valid:
    return 0
  else:
    value = 100
    value = value - len(espacios) * 5 #penalizar numero de huecos
    for hueco in espacios:
      value = value - (1/hueco) * 3
    return clamp_to_zero(value)

In [6]:
def fitnessSinPieza(space):
    espacios = huecos(space)
    value = 100
    value = value - len(espacios) * 5 #penalizar numero de huecos
    for hueco in espacios:
      value = value - (1/hueco) * 3
    return clamp_to_zero(value)

In [7]:
def colocar_pieza(space, pieza, x, y):
    temp = copy.deepcopy(space)  # Deep copy the space to avoid modifying the original
    if len(pieza)+x >= len(space) or len(pieza[0])+y >= len(space[0]):
                return False  # Return False if the piece goes out of bounds
    for i in range(len(pieza)):
        for j in range(len(pieza[0])):
            # Check bounds
            if space[x+i][y+j] == 1:
                return False  # Return False if there's an overlap
            temp[x+i][y+j] = 1  # Place the piece in the temporary space
    # Copy the modified temporary space back to the original space
    for i in range(len(space)):
        for j in range(len(space[0])):
            space[i][j] = temp[i][j]
    return True  # Return True if the piece was successfully placed

In [8]:
#Función para crear la pieza, empezamos creando piezas de un máximo de area de 3x3, para empezar con una versión sencilla del problema, haremos que las piezas inicialmente solo puedan ser cuadrados de 1x1, 2x2 o 3x3.

import random

def generar_pieza():
    # Inicializamos una matriz vacía
    sizex = random.randint(1, 6)
    sizey = random.randint(1, 6)
    pieza = [[1 for column in range(sizex)] for row in range(sizey)]
    return pieza


def imprimir_pieza(pieza:PiezaClase):
    for row in pieza.pieza:
        print("  ".join(map(str, row)))
    print("coordenadas: x:",pieza.x_pos, "y:",pieza.y_pos)
    print()

def imprimir_espacio(space):
    for row in space:
        print("  ".join(map(str, row)))
    print()



# Ejemplo de uso
if __name__ == "__main__":
    pieza = PiezaClase(generar_pieza(),random.randint(1, 6),random.randint(1, 6))
    imprimir_pieza(pieza)

1  1  1
1  1  1
1  1  1
1  1  1
1  1  1
1  1  1
coordenadas: x: 3 y: 1



In [9]:

x = 16
y = 16
space = [[0 for column in range(x)] for row in range(y)]

x_pos = 0
y_pos = 0
#print(x_pos,y_pos)
#pieza = generar_pieza()
pieza2 = PiezaClase(generar_pieza(),0,0)
imprimir_pieza(pieza2)
print(fitness(space, pieza2.pieza, pieza2.x_pos, pieza2.y_pos))


print("inicial: ")
imprimir_espacio(space)
temp_area = copy.deepcopy(space)
array = huecos(temp_area)

print (temp_area)
print (array)

1  1
1  1
coordenadas: x: 0 y: 0

94.98809523809524
inicial: 
1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

[[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

PRIMERA IMPLEMENTACION DE ACO

In [10]:
#Genero el espacio donde colocamos las piezas
x = 16
y = 16
space = [[0 for column in range(x)] for row in range(y)]

#Genero un array de piezas
piezas = []
for i in range(10):
    piezaIndividual = PiezaClase(generar_pieza(),random.randint(0, 15),random.randint(0, 15))
    piezas.append(piezaIndividual)
    
    #imprimir_pieza(piezas[i])

In [19]:
def heuristico(piezaHeu:PiezaClase): #se puede hacer un heuristico que varíe segun avanza.
    size = len(piezaHeu.pieza)*len(piezaHeu.pieza[0])
    
    return size / (x*x)

In [41]:
def calculoProbabilidades(pheromonesCopy, piezas):
    alpha = 0.5
    beta = 0.5
    dividendo = []
    suma = 0
    for i in range(0, len(piezas)):
        dividendo.append(pow(pheromonesCopy[i],alpha) * pow(heuristico(piezas[i]),beta))
        suma += dividendo[i]
    for j in dividendo:
        j = j/suma
    print("Dividendo",dividendo)
    return dividendo

In [37]:
#El array sol contiene n arrays, donde n es len de piezas, en los cuales el primer valor es
# si se pone o no se pone le pieza y los otros dos la posicion
# En caso de que se ponga se intenta colocar por ahora de manera aleatoria 50 veces, si no se consigue se pone como False
# esto despues se hara con feromonas y tal pero por ahora random
def generar_sol(space, pheromones):
    sol = [[0 for column in range(0,3)] for row in range(len(piezas))]
    probabilidades=calculoProbabilidades(copy.deepcopy(pheromones), piezas)
    for i in range(len(piezas)):
        if(random.random() < probabilidades[i]):
            sol[i][0] = True
        else:
            sol[i][0] = False
        if (sol[i][0] == True):
            for j in range (0,50):
                posTempX = random.randint(0, 15)
                posTempY = random.randint(0, 15)
                if(fitness(space,piezas[i].pieza,posTempX, posTempY) != 0):
                    sol[i][1] = posTempX
                    sol[i][2] = posTempY
                    break
                elif(j == 10):
                    sol[i][0] = False
                    sol[i][1] = -1
                    sol[i][2] = -1
        else:
            sol[i][1] = -1
            sol[i][2] = -1
    return sol

# print(generar_sol(space))
# imprimir_espacio(space)
# print(fitnessSinPieza(space))

In [43]:
#Genero el ACO
pheromones = [random.random()+0.1 for i in range(0,len(piezas))]
best_solution = []
numberAnts = 10
for i in range(0,1):
    caminos = []
    for j in range(numberAnts):
        solution = generar_sol(copy.deepcopy(space), pheromones)
        print(fitnessSinPieza(solution), solution) #TO DO, imprimir space con esta sol para leerlo mejor
        caminos.append(solution)
        fit = fitnessSinPieza(solution)
        if(fit > fitnessSinPieza(best_solution)):
            best_solution = solution
    

imprimir_espacio(space)

Dividendo [0.848744856090167, 0.7554183005516651, 0.9009182090894585, 0.9305049618980126, 0.32048704233440206, 0.8182094248712142, 1.0230558747735, 0.439617847136725, 0.9484650940535244, 0.8346878378705912]
69.5 [[False, -1, -1], [True, 11, 7], [True, 0, 11], [True, 8, 1], [True, 2, 1], [False, -1, -1], [True, 1, 4], [True, 6, 11], [False, 5, 4], [False, -1, -1]]
Dividendo [0.848744856090167, 0.7554183005516651, 0.9009182090894585, 0.9305049618980126, 0.32048704233440206, 0.8182094248712142, 1.0230558747735, 0.439617847136725, 0.9484650940535244, 0.8346878378705912]
70.25 [[True, 12, 7], [True, 0, 7], [True, 6, 9], [True, 6, 5], [True, 5, 1], [False, -1, -1], [True, 11, 0], [True, 11, 8], [False, -1, -1], [False, 0, 0]]
Dividendo [0.848744856090167, 0.7554183005516651, 0.9009182090894585, 0.9305049618980126, 0.32048704233440206, 0.8182094248712142, 1.0230558747735, 0.439617847136725, 0.9484650940535244, 0.8346878378705912]
72.5 [[True, 8, 7], [True, 11, 0], [True, 7, 1], [True, 8, 8], 

In [2]:
#ACO template

class Piece:
    def __init__(self, x_dim, y_dim):
        self.x_dim = x_dim
        self.y_dim = y_dim

class Choice:
    def __init__(self, pieza, x_pos, y_pos):
        self.piece = pieza
        self.x_pos = x_pos
        self.y_pos = y_pos

class Solution:
    def __init__(self, x_dim, y_dim):
        self.x_dim = x_dim
        self.y_dim = y_dim
        self.board = [[0 for column in range(x_dim)] for row in range(y_dim)]

    def does_Choice_fit(self, choice):
        if choice.piece.x_dim + choice.x_pos > self.x_dim:
            return False
        if choice.piece.y_dim + choice.y_pos > self.y_dim:
            return False
        for i in range(choice.piece.x_dim):
            for j in range(choice.piece.y_dim):
                if self.board[choice.x_pos + i][choice.y_pos + j] == 1 :
                    return False
        return True

    def place_Choice(self, choice):
        for i in range(choice.piece.x_dim):
            for j in range(choice.piece.y_dim):
                self.board[choice.x_pos + i][choice.y_pos + j] = 1


def initialize_pheromones():
    Pheromones = ??
    return Pheromones

def update_pheromones(Pheromones, Solution_list):
    New_Pheromones = initialize_pheromones()
    ??
    return New_Pheromones

def evaluate_fitness(solution):
    fitness = 0
    ??
    return fitness

def get_candidates(Solution):
    Candidates = []
    ??
    return Candidates

def heuristic(choice):
    return ??


def calculate_probabilities(Candidates, Pheromones):
    alpha = 0.5
    beta = 0.5
    dividend = []
    sum = 0
    for i in range(0, len(Candidates)):
        dividend.append(pow(Pheromones[i],alpha) * pow(heuristic(Candidates[i]),beta))
        sum += dividend[i]
    for j in dividend:
        j = j/sum
    return dividend

def random_choice(Candidates, Probabilities):
    for i in range(len(Candidates)):
        if random.random() < Probabilities[i]: #good option for experimenting an using other choosing algos
            return Candidates[i]

def construct_solution(Pheromones):
    Solution = []
    Candidates = get_candidates(Solution) #candidates are always valid
    while (not Candidates.empty()):
        Probabilities = calculate_probabilities(Candidates, Pheromones)
        choice = random_choice(Candidates, Probabilities)
        Solution.place_Choice(choice)
        Candidates = get_candidates(Solution)

    return Solution

def ACO(n_ants):
    Pheromones = initialize_pheromones()
    best_fitness = 0
    best_solution = []
    while (not has_finished()):
        Solution_list = []
        for i in range(n_ants):
            solution = construct_solution(Pheromones)
            Solution_list.append(solution)
            fintess = evaluate_fitness(solution)
            if (fintess > best_fitness):
                best_solution = solution
                best_fitness = fitness

        Pheromones = update_pheromones(Pheromones, Solution_list)

    return best_solution





#
#
#
#
#
#
#
#
#
#
#
#
#

SyntaxError: invalid syntax (3729682758.py, line 38)